In [1]:
#visualisation and work with dfs
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

from keras.models import Model, Sequential
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, Bidirectional, TimeDistributed, Flatten
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping

%matplotlib inline

Using TensorFlow backend.


# Data preparations

In [2]:
#Load the data into Pandas dataframe
data2 = pd.read_csv('../task2_data/task2_en_training.tsv', sep='\t')[['class', 'tweet']]
data3 = pd.read_csv('../task3_data/task3_training.tsv', sep='\t')[['type', 'tweet']]

# Vectorization by keras

# Vectorization by fasttext

In [3]:
import fasttext

max_words = 10000
max_len = 50


data2 = pd.read_csv('../task2_data/task2_en_training.tsv', sep='\t')[['class', 'tweet']]
df = data2[:5000]

df_ft = df.iloc[np.random.permutation(len(df))]
df_ft['class'] = df_ft['class'].apply(lambda x: '__label__' + str(x))

df_train, df_test = train_test_split(df_ft , test_size=0.2)

df_train[['class', 'tweet']].to_csv('TRAIN.txt', header=False, index=False, sep="\t")
df_test[['class', 'tweet']].to_csv('TEST.txt', header=False, index=False, sep="\t")
#print(df_train, df_test)

graph = []
maxima, maxima_idx = 0, 0

# просто отрабатывает модель с классификацией, но я просто забираю потом отсюда вектора слов 
for epoch in range(15):
    model = fasttext.train_supervised(input='TRAIN.txt', epoch=epoch)
    All, Pos = 0, 0
    for (clas, tweet) in zip(df_test['class'], df_test['tweet']):
        All += 1
        if int(clas[-1]) == int(model.predict(tweet, k=2)[0][0][-1]): 
            #print(int(clas[-1]),int(model.predict(tweet, k=2)[0][0][-1]))
            Pos += 1
    graph.append(Pos/All)
    if Pos/All > maxima: maxima, maxima_idx = Pos/All, epoch

In [4]:
#Split into training and test data.
X_train = df_train['tweet']
X_test = df_test['tweet']
y_train = df_train['class']
y_test = df_test['class']

y_train = y_train.apply(lambda x: x[-1])
y_test = y_test.apply(lambda x: x[-1])

# it's special. we can't work with other format
X_train = np.array(X_train)
X_test = np.array(X_test)

#по сути на этом этапе можно слегка почистить составляющие твитов
for i in range(len(X_train)):
    sentence = X_train[i].split(' ')
    sentence = list(map(lambda x: model.get_word_vector(x), sentence))
    X_train[i] = sentence

for i in range(len(X_test)):
    sentence = X_test[i].split(' ')
    sentence = list(map(lambda x: model.get_word_vector(x), sentence))
    X_test[i] = sentence

maxima = 0
for ar in X_train:
    if len(ar)>maxima: maxima = len(ar)
for ar in X_test:
    if len(ar)>maxima: maxima = len(ar)
max_len = maxima + 1
X_train.shape

(4000,)

In [5]:
pd.DataFrame(X_train.tolist())

,0,1,2,3,4,5,6,7,8,9,...,52,53,54,55,56,57,58,59,60,61
0,"[0.007326942, -0.001178352, -0.0044796197, 0.0...","[-0.0050803577, -0.04154915, 0.018837588, 0.01...","[0.0019437422, -0.046137523, 0.01817717, 0.031...","[-0.0053896075, 0.16922757, -0.06386226, -0.07...","[0.009889823, 0.064105675, -0.015830984, -0.03...",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,"[0.0016800871, -0.010308716, 0.014017366, -0.0...","[-0.0091238385, 0.044963453, -0.008846778, -0....","[-0.0049657, 0.050176114, -0.016427044, -0.021...","[0.0054265535, 0.0101290215, 0.0019999694, -0....","[0.008678156, -0.026480887, 0.0050281254, 0.01...","[0.0016800871, -0.010308716, 0.014017366, -0.0...","[-0.0091238385, 0.044963453, -0.008846778, -0....","[0.006070142, -0.0016596343, -0.004112491, -0....","[-0.006737003, 0.0072718244, 0.0002808559, 0.0...","[-0.002847189, 0.009732389, -0.011699217, -0.0...",...,None,None,None,None,None,None,None,None,None,None
2,"[-0.0066197934, 0.03318502, -0.007817613, -0.0...","[0.0052956557, 0.041752048, -0.014673589, -0.0...","[-0.00025526638, 0.020293687, -0.0019060838, -...","[0.0052070078, 0.06090359, -0.02629009, -0.036...","[0.008846447, 0.04917074, -0.010107023, -0.027...","[0.0038174218, -0.020817686, 0.014370998, 0.01...","[0.006578174, 0.002919121, -0.0032376235, -0.0...","[0.0075373743, 0.042334937, -0.023268014, -0.0...","[0.002773837, -0.04247997, 0.0147432685, 0.025...","[0.008846447, 0.04917074, -0.010107023, -0.027...",...,None,None,None,None,None,None,None,None,None,None
3,"[0.008805466, 0.03158783, -0.007409056, -0.020...","[-0.0038769068, 0.016522454, -0.005856807, 0.0...","[0.0059538055, -0.020897072, 0.0020432882, 0.0...","[-0.008345177, -0.009852273, 0.010177472, 0.00...","[-0.005498815, -0.00054521614, -0.0027136737, ...","[-0.003119269, 0.05266913, -0.0090858275, -0.0...","[-0.00040576915, 0.010545197, -0.004323791, -0...","[0.0034489369, -0.01774719, 0.010491881, 0.014...","[0.0038817695, 0.0073486627, -0.009647941, 0.0...","[0.0048995917, 0.016172435, -0.01600366, -0.01...",...,None,None,None,None,None,None,None,None,None,None
4,"[0.004156238, 0.15234856, -0.04992758, -0.0723...","[-0.0035511, 0.01331947, -0.005517294, 0.00111...","[0.009184993, 0.006251867, 0.004026371, -0.000...","[-0.00085588393, 0.007651745, 0.0068408893, -0...","[-0.0006877813, 0.068153836, -0.022919975, -0....","[0.004190488, 0.024208102, -0.005177064, -0.00...","[0.008814865, 0.0059632114, 0.004903684, -0.00...","[-0.008750098, 0.07515895, -0.032534502, -0.05...","[3.357938e-05, 0.019247252, 0.0022100527, -0.0...","[-6.583927e-05, -0.055574752, 0.020937165, 0.0...",...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,"[-0.0056000734, 0.019416526, -0.013192286, -0....","[0.0051625045, 0.0022792318, -0.0024613673, -0...","[0.0059538055, -0.020897072, 0.0020432882, 0.0...","[-0.00076128554, 0.03061755, -0.014585061, -0....","[0.0032251154, 0.014642637, -0.00775953, -0.00...","[-0.0018732751, -0.052057855, 0.015633762, 0.0...","[-0.0042026676, -0.041151505, 0.018696168, 0.0...","[0.0024007582, 0.018977832, -0.0012626062, -0....","[-0.0047335294, -0.0047997874, 0.006472804, 0....","[-0.004171299, 0.014805177, -0.01212997, -0.00...",...,None,None,None,None,None,None,None,None,None,None
3996,"[0.008205905, -0.0022202092, -0.0028565254, -0...","[0.006711111, 0.015761878, -0.005382852, -0.00...","[0.008846447, 0.04917074, -0.010107023, -0.027...","[0.0061258147, 0.10621651, -0.039967407, -0.05...","[0.006390969, 0.0034472935, -0.010131744, -0.0...","[-0.0019761417, 0.037326865, -0.0036062337, -0...","[0.00622613, 0.06388563, -0.029853532, -0.0310...","[0.0034489369, -0.01774719, 0.010491881, 0.014...","[0.0036529547, 0.09087309, -0.03500909, -0.051...","[0.006070142, -0.0016596343, -0.004112491, -0....",...,None,None,None,None,None,None,None,None,None,None
3997,"[-0.0070633776, 0.02716172, 0.00050071935, -0...

In [6]:
#ars = [[1,2,3], [4,5,6,7], [9,1]]

def add_pads(ars, max_len):
#     lenn = 0
#     for ar in ars: 
#         if len(ar)>lenn: lenn = len(ar)

    for i in range(len(ars)):
        while len(ars[i]) < max_len: ars[i].insert(0,np.array([0 for i in range(100)]))
    return ars

X_train = add_pads(X_train, max_len)
X_test = add_pads(X_test, max_len)

In [7]:
X_train = X_train.tolist()
pd.DataFrame(X_train)

,0,1,2,3,4,5,6,7,8,9,...,53,54,55,56,57,58,59,60,61,62
0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.007326942, -0.001178352, -0.0044796197, 0.0...","[-0.0050803577, -0.04154915, 0.018837588, 0.01...","[0.0019437422, -0.046137523, 0.01817717, 0.031...","[-0.0053896075, 0.16922757, -0.06386226, -0.07...","[0.009889823, 0.064105675, -0.015830984, -0.03..."
1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",...,"[-0.001470459, -0.0480298, 0.010239306, 0.0247...","[-0.00904925, -0.008111593, 0.005251076, -0.01...","[0.008678156, -0.026480887, 0.0050281254, 0.01...","[0.0065176873, 0.010030612, -0.012567232, -0.0...","[0.0015943439, -0.004177256, -0.0011759811, 0....","[-0.0049657, 0.050176114, -0.016427044, -0.021...","[0.0012314506, -0.0012272104, -0.009665569, -0...","[0.0068099396, -0.033500433, 0.022158831, 0.01...","[-0.0049657, 0.050176114, -0.016427044, -0.021...","[0.0022122583, -0.106067985, 0.035101958, 0.05..."
2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",...,"[-0.00025526638, 0.020293687, -0.0019060838, -...","[0.0052070078, 0.06090359, -0.02629009, -0.036...","[0.008846447, 0.04917074, -0.010107023, -0.027...","[0.0038174218, -0.020817686, 0.014370998, 0.01...","[0.006578174, 0.002919121, -0.0032376235, -0.0...","[0.0075373743, 0.042334937, -0.023268014, -0.0...","[0.002773837, -0.04247997, 0.0147432685, 0.025...","[0.008846447, 0.04917074, -0.010107023, -0.027...","[0.0060552377, 0.028245214, -0.011929116, -0.0...","[0.009350851, 0.0065019247, 0.0029838127, -0.0..."
3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",...,"[0.0048995917, 0.016172435, -0.01600366, -0.01...","[0.0077279243, 0.056181103, -0.023450984, -0.0...","[-0.005362019, -0.040449064, 0.010139206, 0.02...","[-0.0038740707, -0.046371486, 0.020311236, 0.0...","[-0.00713276, -0.08821718, 0.0309349, 0.034005...","[-0.00076128554, 0.0

In [8]:
X = np.array(X_train).reshape(len(X_train), len(X_train[1]), 100)

In [9]:
X.shape

(4000, 63, 100)

In [10]:
X_train = np.array(X_train)

# Configure LSTM

In [43]:
# configure network
#n_batch = len(X) # !!!!!
n_batch = 50
n_epoch = 10
n_neurons = 600

# design network
model = Sequential()
model.add(LSTM(n_neurons, batch_input_shape=(n_batch, X_train.shape[1], X_train.shape[2]), 
               stateful=True, dropout=0.5))
model.add()
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', 
              optimizer='rmsprop', 
              metrics=['acc'])


model.summary()

TypeError: add() missing 1 required positional argument: 'layer'

# Fit models

In [12]:
# fit network
for i in range(n_epoch):
    model.fit(X_train, y_train, epochs=1, batch_size=n_batch, verbose=1, shuffle=False)
    model.reset_states()

Epoch 1/1
4000/4000 [==============================] - 11s 3ms/step - loss: 0.1893 - acc: 0.9517
Epoch 1/1
4000/4000 [==============================] - 10s 3ms/step - loss: 0.0607 - acc: 0.9818
Epoch 1/1
4000/4000 [==============================] - 10s 3ms/step - loss: 0.0525 - acc: 0.9870
Epoch 1/1
4000/4000 [==============================] - 10s 2ms/step - loss: 0.0386 - acc: 0.9890
Epoch 1/1
4000/4000 [==============================] - 10s 2ms/step - loss: 0.0343 - acc: 0.9893
Epoch 1/1
4000/4000 [==============================] - 10s 2ms/step - loss: 0.0801 - acc: 0.9837
Epoch 1/1
4000/4000 [==============================] - 10s 2ms/step - loss: 0.0383 - acc: 0.9883
Epoch 1/1
4000/4000 [==============================] - 10s 2ms/step - loss: 0.0422 - acc: 0.9883
Epoch 1/1
4000/4000 [==============================] - 10s 2ms/step - loss: 0.0404 - acc: 0.9890
Epoch 1/1
4000/4000 [==============================] - 10s 2ms/step - loss: 0.0310 - acc: 0.9902


In [13]:
# аналогично X_train подготовлен X_test
X_test = X_test.tolist()
X_test = np.array(X_test).reshape(len(X_test), len(X_test[1]), 100)
X_test = np.array(X_test)
X_test.shape
type(X_test)

numpy.ndarray

In [14]:
# перекидываем веста в новую модель для размера батча 1
n_batch = 1

# re-define model
new_model = Sequential()
new_model.add(LSTM(n_neurons, batch_input_shape=(n_batch, X_train.shape[1], X_train.shape[2]), stateful=True))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# copy weights
old_weights = model.get_weights()
new_model.set_weights(old_weights)

# compile model
new_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['acc'])

In [38]:
y_train

1591    0
3820    0
4008    0
3792    0
379     0
       ..
4242    0
284     0
1299    0
3893    0
2655    0
Name: class, Length: 4000, dtype: object

# Evaluate

In [31]:
for i in range(len(X)):
    testX, testy = np.array(X_train.tolist()[i]), np.array(y_test.tolist()[i])
    testX = testX.reshape(1, 63, 100)
    yhat = new_model.predict(testX, batch_size=1)
    print(f'>Expected={y_test.tolist()[i]}, Predicted={yhat}')

>Expected=0, Predicted=[[-0.1400146  -0.14805388  0.5172186  -0.03970669  0.07259989  0.12059539
  -0.0020032  -0.11736003 -0.08701103  0.0977805   0.08564936 -0.38464174
   0.31448293  0.14389302  0.08139694  0.16669747 -0.14257038 -0.17615591
  -0.1261392  -0.07018367  0.06027267  0.3031799   0.23920162  0.24727124
  -0.35538468  0.07426848  0.13096109  0.04957148  0.3490798   0.19279362
   0.23694286  0.38335878  0.0789171   0.13763338 -0.71840596  0.18472005
  -0.17723563  0.293651    0.04977634  0.02328567 -0.14266466  0.03465788
   0.11702342 -0.16940783  0.11977962 -0.13278434 -0.21240465  0.6718441
   0.14781654 -0.05932437 -0.5098247  -0.06812666  0.2123222   0.08260628
  -0.19175704 -0.21413767 -0.05694979 -0.07050747 -0.11166658  0.18075182
   0.6624843  -0.36548707  0.04795307 -0.07549742  0.18081751 -0.16068025
  -0.19365074 -0.10368768  0.6538362  -0.09519664  0.18791525 -0.09730059
   0.13572958 -0.26854303  0.05098645  0.37905908 -0.6200455   0.0583361
  -0.17834495 -0.

>Expected=1, Predicted=[[-5.91434026e-03 -5.28989919e-02  5.15148938e-01 -2.56543756e-02
   2.40291972e-02  3.12094670e-02 -1.19791017e-04 -1.22950643e-01
  -3.54253948e-02  2.86599062e-03  1.96664780e-02 -6.26896620e-01
   5.78665376e-01  2.57928986e-02  1.29031688e-01  6.72891140e-02
  -1.27721146e-01 -1.16863243e-01 -6.75178766e-02 -3.22581343e-02
   4.35380973e-02  4.49714005e-01  6.98336288e-02  1.21184088e-01
  -5.08370161e-01  3.37817743e-02  8.74832422e-02  2.95474064e-02
   5.66107690e-01  1.85101271e-01  1.24451011e-01  4.92080927e-01
   2.46111546e-02  1.23254284e-01 -7.27581084e-01  1.58074334e-01
  -1.34079769e-01  4.79231566e-01  3.26941200e-02  8.99282191e-03
  -2.32598130e-02 -1.17358090e-02  1.13694027e-01 -2.44833022e-01
   4.37062755e-02 -1.03632353e-01 -3.49112868e-01  6.89320266e-01
   1.34435833e-01 -2.06770748e-02 -8.46816957e-01 -1.49521558e-02
   1.30459681e-01  8.04673657e-02 -2.72352457e-01 -1.73514828e-01
  -1.58035159e-01 -1.63135886e-01 -6.98280260e-02  2.

>Expected=1, Predicted=[[ 3.24692857e-03  6.16508862e-03  5.19330382e-01 -2.61664540e-02
   3.14275138e-02  1.25143630e-02 -1.67177676e-03 -4.58012968e-02
  -5.72287031e-02 -2.45079887e-03  4.42869253e-02 -5.16890168e-01
   4.67232347e-01  1.37055302e-02  1.00400761e-01  8.33615139e-02
  -1.01157904e-01 -3.62441950e-02 -1.24244340e-01 -5.04142940e-02
   3.37581858e-02  3.68396491e-01  8.12883824e-02  2.19449848e-02
  -4.00263667e-01  3.31747171e-04  6.48414865e-02  2.15671901e-02
   5.26037216e-01  1.03417359e-01  7.31024370e-02  3.34836900e-01
   1.09906867e-02  3.64918858e-02 -6.44392848e-01  8.43454376e-02
  -1.04645289e-01  2.17388257e-01  4.04019840e-02  8.57760012e-03
  -3.23619731e-02 -1.13601601e-02  4.78255153e-02 -2.02331096e-01
   1.27946232e-02  1.93142320e-03 -8.00087675e-02  6.35400534e-01
   1.66409314e-01 -1.27074355e-02 -6.69047415e-01 -8.06964468e-03
   4.89632636e-02  5.75083047e-02 -2.11099610e-01 -2.01761276e-01
  -1.15715213e-01 -5.91299869e-02 -5.31535335e-02  7.

>Expected=0, Predicted=[[ 0.05380597  0.21233061 -0.2791594  -0.01516806 -0.15890633 -0.21234994
   0.00715058  0.2649826   0.26126656 -0.04719216 -0.22193678  0.18482643
  -0.08514145 -0.14795604 -0.02354648 -0.48607102  0.21523829  0.338775
   0.55334955  0.24699533 -0.06690763 -0.05756475 -0.0938204  -0.07523003
   0.16198035 -0.08971141 -0.21077277 -0.0604733  -0.2632349  -0.175492
  -0.12902425 -0.11816834 -0.13061136 -0.13940819  0.15680315 -0.07540181
   0.10631991 -0.1733215  -0.12015726 -0.19648948  0.42143333  0.07298391
  -0.07280913  0.18125421 -0.08625288  0.25737795  0.09870067 -0.2264287
  -0.41594267  0.03405477  0.12512246  0.03326852 -0.1763092  -0.07262775
   0.37226883  0.37093037  0.05850596  0.33083314  0.19500834 -0.16265121
  -0.11598036  0.17166212 -0.03145491  0.08370982 -0.02531747  0.28047177
   0.24326757  0.36184502 -0.1024389   0.07451663 -0.57719195  0.2067736
  -0.10868192  0.29485774 -0.20770034 -0.09680892  0.22988333  0.00916009
   0.23779756  0.0227

>Expected=0, Predicted=[[ 9.16598178e-03  9.22944993e-02  3.96572053e-01 -2.68881060e-02
   4.01927233e-02 -2.61814613e-03 -5.77760686e-04  3.28620747e-02
  -6.08803630e-02  1.27674267e-02  1.13329850e-01 -2.67283350e-01
   3.17238778e-01  2.80846134e-02  4.65093963e-02  6.72492534e-02
  -1.88452806e-02  9.99490991e-02 -2.04017684e-01 -7.49406293e-02
   2.69739665e-02  2.29644552e-01  1.27613902e-01 -5.10685258e-02
  -1.68362349e-01 -2.08194815e-02  3.12934518e-02  1.25744492e-02
   2.29161471e-01  2.80706980e-03  1.44658086e-03  7.73549378e-02
  -1.58376433e-02 -5.97028770e-02 -4.88648444e-01  3.51002999e-03
  -2.80724894e-02 -6.96571991e-02  5.56196347e-02  3.77452606e-03
  -2.50385068e-02 -5.02834376e-03 -4.77536023e-03 -8.67199153e-02
  -1.34594860e-02  1.19794913e-01  9.79761407e-02  5.32075763e-01
   1.94665000e-01 -1.54497242e-02 -1.49510145e-01  4.46101977e-03
  -6.35785833e-02  4.31025401e-02 -7.81134367e-02 -1.66915908e-01
  -5.50897457e-02  1.05319701e-01 -1.90663524e-02 -3.

...

>Expected=0, Predicted=[[-0.00383613 -0.04167316  0.4958115  -0.0173734   0.02363131  0.01710498
  -0.00256988 -0.04533432 -0.05083926  0.00279767  0.04458753 -0.49093068
   0.42523348  0.01017682  0.07059212  0.10521007 -0.095102   -0.04177054
  -0.11109893 -0.05066661  0.03146394  0.32061172  0.11876713  0.06982758
  -0.36428002  0.00401153  0.06907858  0.01718274  0.47983152  0.10978767
   0.09627736  0.29996276  0.00888741  0.04452646 -0.6377994   0.08573256
  -0.10106266  0.19387114  0.03340646  0.00831011 -0.03881746 -0.00669117
   0.05306582 -0.16856201  0.02617017 -0.02410543 -0.08896989  0.62173665
   0.16411535 -0.00582414 -0.6130961  -0.00803019  0.08016828  0.05609726
  -0.21844709 -0.20728406 -0.0664148  -0.0774141  -0.05663802  0.07696918
   0.73625594 -0.5243095  -0.00334545  0.01023811  0.6018509  -0.07235996
  -0.25933108 -0.06175243  0.72061795 -0.00816301  0.13908191 -0.05332434
  -0.04328616 -0.24020298  0.01908736  0.43629012 -0.67029357 -0.04706316
  -0.11162964 -

>Expected=0, Predicted=[[-8.33795592e-03 -8.27904195e-02  4.86203521e-01 -1.63707118e-02
   2.19734479e-02  3.39002609e-02 -2.46686116e-03 -8.60752612e-02
  -3.86998691e-02  6.99465908e-03  3.07978224e-02 -5.30470788e-01
   4.15752918e-01  2.48979144e-02  7.46367201e-02  1.00109234e-01
  -1.09477162e-01 -9.81631353e-02 -7.70178735e-02 -4.37758081e-02
   4.14977483e-02  3.14260125e-01  1.22359499e-01  1.58378243e-01
  -4.09022540e-01  2.75583435e-02  8.65915567e-02  2.41853017e-02
   4.87584829e-01  1.46837831e-01  1.53161153e-01  3.69438231e-01
   1.81689989e-02  8.75767693e-02 -6.91848278e-01  1.18618488e-01
  -1.14497893e-01  3.34260195e-01  2.48421989e-02  9.31451842e-03
  -3.49229276e-02 -5.58471074e-03  8.98135006e-02 -1.66646898e-01
   4.58689891e-02 -9.72971246e-02 -2.22723812e-01  6.49684787e-01
   1.35154024e-01 -1.41208023e-02 -6.86574996e-01 -1.80702992e-02
   1.38405502e-01  5.98304868e-02 -2.47916579e-01 -1.78697854e-01
  -6.17080182e-02 -1.33187547e-01 -7.08075389e-02  1.

>Expected=1, Predicted=[[-1.88768599e-02 -4.21714634e-02  5.12345076e-01 -2.35805511e-02
   3.33704017e-02  2.76357587e-02 -3.47101758e-03 -5.94448857e-02
  -5.58182783e-02  5.98707935e-03  4.72549610e-02 -5.01026213e-01
   4.29929793e-01  2.74449028e-02  8.87291580e-02  1.03395656e-01
  -1.05827630e-01 -6.94876239e-02 -1.14666440e-01 -5.32613881e-02
   3.70864384e-02  3.39531243e-01  9.79858786e-02  6.71900883e-02
  -3.86765033e-01  1.11992145e-02  7.70214722e-02  2.09828820e-02
   4.91997212e-01  1.23138607e-01  1.00415595e-01  3.36942255e-01
   1.89463999e-02  4.94315885e-02 -6.58775449e-01  9.75302681e-02
  -1.09514959e-01  2.41044238e-01  3.60157117e-02  9.54804663e-03
  -3.84659320e-02 -7.74896471e-03  5.90361170e-02 -1.89572141e-01
   3.74747813e-02 -3.86046097e-02 -1.27153471e-01  6.38019443e-01
   1.61382526e-01 -1.17417034e-02 -6.44773126e-01 -1.45799825e-02
   8.26559663e-02  5.93093038e-02 -2.20006242e-01 -2.09760517e-01
  -8.59729871e-02 -8.65379125e-02 -6.35950267e-02  9.

>Expected=0, Predicted=[[ 0.00887807  0.2639551  -0.2627748  -0.00942242 -0.15540944 -0.17142253
  -0.02766594  0.23617408  0.3110044  -0.00125531  0.00333225  0.1694342
  -0.06923094 -0.0797659  -0.01829253 -0.4396496   0.21195178  0.398226
   0.45696995  0.20846467 -0.02589477 -0.03178584 -0.00374851 -0.06677514
   0.1533264  -0.06902923 -0.18782718 -0.05764412 -0.28513983 -0.1700262
  -0.13749413 -0.12180922 -0.1333297  -0.22781506  0.13309078 -0.07723764
   0.09711851 -0.1746     -0.04504871 -0.16180353  0.4953255   0.07272198
  -0.07238646  0.18248223 -0.04350757  0.28161243  0.08963367 -0.17063043
  -0.2648178   0.00062731  0.12289706  0.08111687 -0.23489685 -0.02320325
   0.36803898  0.2955896   0.08588236  0.33938074  0.1821667  -0.15428308
  -0.11107562  0.15154679 -0.01469103  0.09174934 -0.03486289  0.25421128
   0.2392703   0.4082586  -0.08077049  0.06100677 -0.5000332   0.18309705
  -0.09184152  0.23367535 -0.21184048 -0.07147376  0.24003975  0.01452509
   0.2734165  -0.03

KeyboardInterrupt: 

In [15]:
scores = new_model.evaluate(X_test, y_test)
print("Accuracy: %.2f%%" % (scores[1]*100))

ValueError: Error when checking target: expected lstm_2 to have shape (300,) but got array with shape (1,)

In [ ]:
# for i, j in zip(model.predict(test_sequences_matrix), Y_test):
#     print(i, j)
list(model1.predict(X_test))
predicts = [i[0] for i in model1.predict(X_test)]

gap = (max(predicts)-min(predicts))/2
for i in range(len(predicts)):
    if i <= gap: predicts[i] = 0
    else: predicts[i] = 1

from sklearn.metrics import classification_report
print(classification_report(y_test, predicts))
      

#f1_score(Y_test, predicts, average='micro')

In [ ]:
# Evaluate the model on the test set
accr = model.evaluate(test_sequences_matrix, Y_test)